### ***Importando as bibliotecas***

In [2]:
from datasets import load_dataset
from unsloth import FastLanguageModel

Unsloth: Your Flash Attention 2 installation seems to be broken?
A possible explanation is you have a new CUDA version which isn't
yet compatible with FA2? Please file a ticket to Unsloth or FA2.
We shall now use Xformers instead, which does not have any performance hits!
We found this negligible impact by benchmarking on 1x A100.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


### ***Inicializando o Foundation Model***

O Foundation Model escolhido para o treinamento será o **unsloth/tinyllama-bnb-4bit** por conta do tamanho e a utilização da memória.

In [3]:
max_seq_length = 256 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/tinyllama-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.43.3.
   \\   /|    GPU: NVIDIA GeForce RTX 4070 Ti. Max memory: 11.994 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


### ***Pré-processamento do dataset***

In [4]:
EOS_TOKEN = tokenizer.eos_token

**Prompt**

Para estruturar os dados de treinamento vai ser utilizado o Alpaca, com isso vai ser possível fornecer ao modelo uma estrutura consistente que vai incluir:
- **Instrução**
- **Input (Contexto adicional)**
- **Resposta esperada**

In [5]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

**Funções de apoio**

As funções abaixo serão utilizadas para o pre-processamento do dataset, uma função valida a quantidade de vazios dentro de um campo especifico e a outra vai formatar os dados do dataset para o prompt.

In [6]:
# Verifica a quantidade de vazio em um campo específico do dataset
def contar_campos_vazios(dataset, campo):
    qtd_vazio = 0

    for example in dataset:
        if not example[campo]:
            qtd_vazio += 1

    return qtd_vazio

In [7]:
# Formata o dataset para o formato esperado pelo modelo
def formatting_prompts_func(examples):
    inputs = examples["title"]
    outputs = examples["content"]
    texts = []
    
    for input, output in zip(inputs, outputs):       
        instruction = "Generate a detailed product description based on the following title:"
        
        # Formata o prompt incluindo o token EOS ao final
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    
    return {"text": texts}

**Formatando o dataset**

Para a formatação será utilizado as duas funções acima.

Inicialmente será validado quantos campos vazios existem na coluna 'content' e na coluna 'title' do dataset, depois será removido esses vazios e finalmente será realizado a formatação dos dados e a criação de uma nova coluna chamada 'text' que vai conter o prompt que será utilizado na avaliação.

In [8]:
# Carrega o dataset completo.
dataset = load_dataset("json", data_files='./LF-Amazon-1.3M/teste/tst.json', split="train")

# Valida a quantidade de linhas com 'content' vazio
qtd_vazio = contar_campos_vazios(dataset, 'content')
print(f"Linhas com content vazio: {qtd_vazio}")

# Valida a quantidade de linhas com 'title' vazio
qtd_vazio = contar_campos_vazios(dataset, 'title')
print(f"Linhas com title vazio: {qtd_vazio}")

# Filtrar o dataset, removendo linhas com 'title' ou 'content' vazios
dataset_processado = dataset.filter(lambda example: example["content"] != "" and example["title"] != "")

# Agora que o dataset está limpo, vamos formatar os prompts
dataset_formatado = dataset_processado.map(formatting_prompts_func, batched=True)

Linhas com content vazio: 323121
Linhas com title vazio: 55556


### ***Separando o dataset***

Para finalizar será pego 30% do dataset de teste e de forma aleatória. 

Os 30% é para que o tempo de avaliação não fique muito grande mas contenha uma quantidade razoável de dados.

In [9]:
# Pegando apenas 30% do meu dataset para avaliação
num_avaliacao = int(len(dataset_formatado) * 0.3)

dataset_teste = dataset_formatado.shuffle(seed=42).select(range(num_avaliacao))

In [10]:
# Salvar o dataset no formato JSON
dataset_teste.to_json('./LF-Amazon-1.3M/teste/tst_formatado.json')

Creating json from Arrow format:   0%|          | 0/180 [00:00<?, ?ba/s]

347026274